Hello dis is test

In [3]:
from Inner_Speech_Dataset.Python_Processing.Data_extractions import  Extract_data_from_subject
%run data_handler.ipynb

datatype = "EEG"
root_dir = "../dataset"
subjectNr = 1

data, label = Extract_data_from_subject(root_dir, subjectNr, datatype)

d, l = load_subject_non_downsampled(2)
print(d.shape)

(600, 128, 4609)


In [23]:
print(type(data[10])) # [39869     3     0     1]  I think it means this:  [unknown     class?     contaminated?     subjectNr]
print(data.shape)
print(data[0].shape)

# 500 trials

<class 'numpy.ndarray'>
(500, 128, 1153)
(128, 1153)


In [24]:
#!ls -la .. # ! to write bash commands

In [99]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.f1 = 16
        self.d = 4
        self.f2 = self.d*self.f1

        # Block 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels = self.f1, kernel_size = (1, 128), padding = 0, bias = False) # 128 = half sampling rate, reduces to 2Hz, 16 is probably F1, the number of filters (output channels)
        self.batchnorm1 = nn.BatchNorm2d(self.f1, False)

        self.conv2 = nn.Conv2d(in_channels = self.f1, out_channels = self.f1*self.d, kernel_size = (128, 1), padding = 0, bias = False) # row, column] 128 = number of sensors for the depth wise convolution
        # I'm missing a maxnorm regilarization here.
        # Add dropout in the forward func.
        self.batchnorm2 = nn.BatchNorm2d(self.f1*self.d, False) # Not sure this is the right number of features.
        self.elu = nn.ELU()
        self.avgPool1 = nn.AvgPool2d((1,4)) # Reduces sampling rate to 64hz

        # Block 2

        # Separable convolution
        self.depthwise = nn.Conv2d(in_channels = self.f1*self.d, out_channels = self.f1*self.d, kernel_size = (1, 32),
                        bias=False, groups = self.f1*self.d, padding = (0, 16//2)) # Captures 500ms of data at sampling rate 64Hz (32 Hz is 500ms of that)
        self.pointwise = nn.Conv2d(in_channels = self.f1*self.d, out_channels = self.f2, kernel_size = 1) # This results in f2 channels of the output here.
        #self.depthwise = nn.Conv2d(4, 4, kernel_size = (1,32), padding=1, groups=4, bias = False) # Captures 500ms of data at sampling rate 64Hz (32 Hz is 500ms of that)
        #self.pointwise = nn.Conv2d(4, self.f2, kernel_size = 1) # This results in f2 channels of the output here, not sure if it's meant to be like this. But I proceede to run a test

        self.batchnorm3 = nn.BatchNorm2d(self.f2, False)

        self.avgPool2 = nn.AvgPool2d((1,8))

        self.classifier = nn.Linear(1920,4, bias = False)
        self.softmax = nn.Softmax()


    def forward(self, x, dropout = 0.5): # Dropout 0.5 dropout for within-subject and 0.25 for cross-subject
        # Block 1
        res = self.conv1(x)
        res = self.batchnorm1(res)
        res = self.conv2(res)
        res = torch.renorm(res, p=2, dim=0, maxnorm=1)
        res = self.batchnorm2(res)
        #res = F.dropout(0.25)
        res = self.elu(res)
        res = self.avgPool1(res)
        res = F.dropout(res, dropout)
        res = self.depthwise(res)
        res = self.pointwise(res)
        res = self.batchnorm3(res)
        res = self.elu(res)
        res = self.avgPool2(res)
        res = F.dropout(res, dropout)
        res = torch.flatten(res)
        res = self.classifier(res)
        res = self.softmax(res)
        return res


In [101]:
network = EEGNet().float()
d = torch.tensor(data[1])
print(d.size())
d = torch.unsqueeze(torch.unsqueeze(d,0),0).float() # Unsqueeze adds a wrapper dimension of 1s
print(d.size())
r = network.forward(d)
#print(r.size()) # [1,4,1,256] [samples, channels, height, width]
# [1,64,3,28] right before flatten leads to 64*3*28 = 5376 dim
print(r)

print(torch.max(r,0))


torch.Size([128, 1153])
torch.Size([1, 1, 128, 1153])
tensor([0.4108, 0.1735, 0.1532, 0.2625], grad_fn=<SoftmaxBackward0>)
torch.return_types.max(
values=tensor(0.4108, grad_fn=<MaxBackward0>),
indices=tensor(0))


/tmp/ipykernel_42372/1427443249.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  res = self.softmax(res)


In [115]:
from braindecode import models
# input_window_samples decides the number of samples in each window (?) for 250Hz they set it to 1000 (4 seconds). Has to be larger than the networks receptive field.
bo = models.EEGNetv4(in_chans = 128, n_classes = 4, F1 = 8, D = 2, F2 = 16, kernel_length = 128, input_window_samples = 1153, drop_prob = 0.5)


d = torch.tensor(data[0]).float()
print(d.shape[1])
d = torch.unsqueeze(d,0).float() # Unsqueeze adds a wrapper dimension of 1s
print(d.size())
r = bo(d)
print(r)
print(torch.argmax(r, dim=1))

1153
torch.Size([1, 128, 1153])
tensor([[-1.4367, -1.3356, -1.3615, -1.4146]], grad_fn=<SelectBackward0>)
tensor([1])


In [117]:
from braindecode import models
# input_window_samples decides the number of samples in each window (?) for 250Hz they set it to 1000 (4 seconds). Has to be larger than the networks receptive field.
bo2 = models.ShallowFBCSPNet(in_chans = 128, n_classes = 4, input_window_samples = 508, drop_prob = 0.5)


d = torch.tensor(data[0]).float()
print(d.shape[1])
d = torch.unsqueeze(d,0).float() # Unsqueeze adds a wrapper dimension of 1s
print(d.size())
r = bo2(d)
print(r.size())
print(torch.argmax(r, dim=0))

1153
torch.Size([1, 128, 1153])
torch.Size([1, 4, 42])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
